# <font color= #99C8F5> **SARIMA Model** </font>

_by Isabel Valladolid, Oscar Rocha & Vivienne Toledo_

22/02/2026.

---

# <font color= #99C8F5> **Introduction** </font>

This notebook will cover a SARIMA model developed for predicting total goals scored by a specific team on the Eredivisie Division (Netherlands). Data was obtained from the Football Data API (https://www.football-data.org/), covering from mid 2023 all the way through the end of 2025.

---

# <font color= #99C8F5> **Libraries & Data** </font>

In [30]:
# General
import numpy as np
import pandas as pd
import requests
from dotenv import load_dotenv
import os

# Visualizations
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [31]:
import statsapi
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime, timedelta
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.stattools import adfuller, acf, pacf

In [32]:
# Get the API KEY
load_dotenv()
API_KEY = os.getenv('API_KEY')

In [33]:
# # Log into the API 
# BASE_URL = "https://api.football-data.org/v4"
# headers = {"X-Auth-Token": API_KEY}

# # Get data from Eredivisie 
# competition_code = "DED"
# params = {
#     "dateFrom": "2025-07-01",           # Jan 1st 
#     "dateTo": "2025-12-31",             # Dec 31st
#     "status": "FINISHED"                # Finished matches
# }

# # Get all matches
# response = requests.get(
#     f"{BASE_URL}/competitions/{competition_code}/matches",
#     headers=headers,
#     params=params
# )
# data = response.json()
# all_matches = data.get("matches", [])

# # Convert to dataframe
# df = pd.DataFrame([{
#     "date": m["utcDate"],
#     "homeTeam": m["homeTeam"]["name"],
#     "awayTeam": m["awayTeam"]["name"],
#     "homeGoals": m["score"]["fullTime"]["home"],
#     "awayGoals": m["score"]["fullTime"]["away"],
#     "winner": m["score"]["winner"]
# } for m in all_matches])

# # Convert date to datetime and sort
# df["date"] = pd.to_datetime(df["date"])
# df.sort_values("date", inplace=True)

# # Save dataframe locally
# df.to_csv("eredivisie_matches_2025.csv", index=False)

# df.head()

In [34]:
# Access data with no API_KEY
df_2023 = pd.read_csv('eredivisie_matches_2023.csv')
df_2024 = pd.read_csv('eredivisie_matches_2024.csv')
df_2025 = pd.read_csv('eredivisie_matches_2025.csv')

# Merge dataframes
df = pd.concat([df_2023, df_2024, df_2025])

In [35]:
df.head()

,date,homeTeam,awayTeam,homeGoals,awayGoals,winner
0,2023-08-11 18:00:00+00:00,FC Volendam,SBV Vitesse,1,2,AWAY_TEAM
1,2023-08-12 14:30:00+00:00,PSV,FC Utrecht,2,0,HOME_TEAM
2,2023-08-12 16:45:00+00:00,SC Heerenveen,RKC Waalwijk,3,1,HOME_TEAM
3,2023-08-12 18:00:00+00:00,AFC Ajax,Heracles Almelo,4,1,HOME_TEAM
4,2023-08-12 19:00:00+00:00,PEC Zwolle,Sparta Rotterdam,1,2,AWAY_TEAM


# <font color= #99C8F5> **Filter Dataset** </font>

The **_AFC Ajax_** team was selected to use for predictions. Thus, filtering the dataset is in order:

In [36]:
team = 'AFC Ajax'

# Convert date column to datetime
df['date'] = pd.to_datetime(df['date'], utc=True)

# Filter matches by team
ajax_matches = df[(df['homeTeam'] == team) | (df['awayTeam'] == team)].copy()

# Add goals scored by team
ajax_matches['goals_scored'] = ajax_matches.apply(
    lambda row: row['homeGoals'] if row['homeTeam'] == team else row['awayGoals'],
    axis=1
)

# Extract the date
ajax_matches['match_date'] = ajax_matches['date'].dt.date
# Sum goals per day
goals_per_day = ajax_matches.groupby('match_date')['goals_scored'].sum().reset_index()
# Add team column
goals_per_day['team'] = team

# Rename columns for clarity
goals_per_day.rename(columns={'match_date': 'date', 'goals_scored': 'total_goals'}, inplace=True)
goals_per_day.head()

,date,total_goals,team
0,2023-08-12,4,AFC Ajax
1,2023-08-19,2,AFC Ajax
2,2023-09-03,0,AFC Ajax
3,2023-09-17,1,AFC Ajax
4,2023-09-27,0,AFC Ajax


# <font color= #99C8F5> **Visualization** </font>

The time series of the daily goals scored by **_AFC Ajax_** in the Eredivisie Division League looks like this:

In [37]:
# Time series visualization
fig = go.Figure()
fig.add_trace(go.Scatter(x=goals_per_day.date, y=goals_per_day.total_goals, mode='lines', name='Goals per Day'))

fig.update_layout(
    title=f'Daily Goals by {team}',
    xaxis_title='Date',
    yaxis_title='Total Goals'
)
fig.show()

As seen above, the time series is quite inconsistent and has multiple "skips", since the **_AFC Ajax_** does not play every single day for the Eredivisie Division League. This results in a choppy time series that may not fit any model perfectly.

# <font color= #99C8F5> **Selecting Model Order** </font>

A SARIMA Model is defined following this structure:

$$
SARIMA(p, d, q)(P, D, Q)_s
$$

To build the SARIMA Model, it's imperative to find the model order for each component.

In [38]:
def check_stationarity(series, title="Serie Original"):
    result = adfuller(series.dropna())
    print(f'ADF Test: {title}')
    print(f'Estadístico ADF: {result[0]:.4f}')
    print(f'p-value: {result[1]:.4f}')
    is_stationary = result[1] < 0.05
    print(f"¿Es estacionaria? {'SÍ' if is_stationary else 'NO'}\n")
    return is_stationary

# 1. Revisamos la serie original
check_stationarity(goals_per_day["total_goals"], "Nivel Original")

# 2. Aplicamos Primera Diferencia (d=1)
ts_mlb_diff = goals_per_day['total_goals'].diff()

# 3. Revisamos la serie diferenciada
check_stationarity(ts_mlb_diff, "Primera Diferencia (d=1)")

# Creamos una figura con 2 columnas (Subplots)
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Serie Original (No Estacionaria)", "Serie Diferenciada (Estacionaria d=1)")
)

# Gráfico 1: Serie Original
fig.add_trace(
    go.Scatter(x=goals_per_day.index, y=goals_per_day['total_goals'], name='Original'),
    row=1, col=1
)

# Gráfico 2: Serie Diferenciada
fig.add_trace(
    go.Scatter(x=ts_mlb_diff.index, y=ts_mlb_diff, name='Diferenciada'),
    row=1, col=2
)

# Ajustes de diseño
fig.update_layout(
    title_text="Comparativa: Efecto de la Diferenciación",
    showlegend=False, # Ocultamos leyenda
    height=500
)

fig.show()

ADF Test: Nivel Original
Estadístico ADF: -8.6279
p-value: 0.0000
¿Es estacionaria? SÍ

ADF Test: Primera Diferencia (d=1)
Estadístico ADF: -6.5906
p-value: 0.0000
¿Es estacionaria? SÍ



Based on the differentiated time series, as well as the tests executed, it seems no differentiation order is necessary. However, the 1st differentiation series standarizes the goals per day value and it seems that a stationality is perceived, thus:

$d=0$

$D=1$

Now, we proceed with the $p, d, P, Q, s$ values. For the non-differentiated series, which defines p, d:

In [72]:
# @title Graficamos ACF y PACF
ts_analysis = goals_per_day.set_index('date')['total_goals']
ts_analysis.index = pd.to_datetime(ts_analysis.index)

# Parámetros
lags = 30
alpha = 0.05 # Nivel de significancia del 95%

# Cálculo de valores ACF y PACF
acf_vals = acf(ts_analysis, nlags=lags, alpha=alpha)[0][1:]
pacf_vals = pacf(ts_analysis, nlags=lags, alpha=alpha)[0][1:]

# Colocamos manualmente el intervalo de confianza para plotly
n = len(ts_analysis)
conf_interval = 1.96 / np.sqrt(n)


fig = make_subplots(rows=2, cols=1,
                    subplot_titles=("Función de Autocorrelación (ACF) - Determina MA(q)",
                                    "Autocorrelación Parcial (PACF) - Determina AR(p)"),
                    vertical_spacing=0.15)

# ACF

fig.add_trace(go.Bar(
    x=list(range(1, lags+1)), y=acf_vals,
    name='ACF', marker_color='rgb(31, 119, 180)', showlegend=False
), row=1, col=1)

# Intervalos de confianza (Sombreado)
fig.add_shape(type="rect",
    x0=0.5, y0=-conf_interval, x1=lags+0.5, y1=conf_interval,
    line=dict(color="rgba(0,0,0,0)"), fillcolor="rgba(0,0,0,0.1)",
    row=1, col=1
)
# Líneas límite
fig.add_hline(y=conf_interval, line_dash="dash", line_color="gray", row=1, col=1)
fig.add_hline(y=-conf_interval, line_dash="dash", line_color="gray", row=1, col=1)

# PACF

fig.add_trace(go.Bar(
    x=list(range(1, lags+1)), y=pacf_vals,
    name='PACF', marker_color='rgb(255, 127, 14)', showlegend=False
), row=2, col=1)

# Intervalos de confianza
fig.add_shape(type="rect",
    x0=0.5, y0=-conf_interval, x1=lags+0.5, y1=conf_interval,
    line=dict(color="rgba(0,0,0,0)"), fillcolor="rgba(0,0,0,0.1)",
    row=2, col=1
)
# Líneas límite
fig.add_hline(y=conf_interval, line_dash="dash", line_color="gray", row=2, col=1)
fig.add_hline(y=-conf_interval, line_dash="dash", line_color="gray", row=2, col=1)


fig.update_layout(
    title='<b>Diagnóstico de Estructura: ACF y PACF</b><br><sup>Serie Diferenciada</sup>',
    template='plotly_white',
    height=700,
    bargap=0.8 # Barras delgadas estilo lollipop
)

# Resaltar lags estacionales (7, 14, 21, 28) con líneas verticales rojas tenues
for i in [7, 14, 21, 28]:
    fig.add_vline(x=i, line_width=1, line_dash="dot", line_color="red", opacity=0.5)

fig.show()

None of the values seem relevant for $p$ & $q$ values :(

For the P, Q values with 1st difference:

In [73]:
ts_analysis = ts_mlb_diff.dropna()

# Parámetros
lags = 30
alpha = 0.05 # Nivel de significancia del 95%

# Cálculo de valores ACF y PACF
acf_vals = acf(ts_analysis, nlags=lags, alpha=alpha)[0][1:]
pacf_vals = pacf(ts_analysis, nlags=lags, alpha=alpha)[0][1:]

# Colocamos manualmente el intervalo de confianza para plotly
n = len(ts_analysis)
conf_interval = 1.96 / np.sqrt(n)


fig = make_subplots(rows=2, cols=1,
                    subplot_titles=("Función de Autocorrelación (ACF) - Determina MA(q)",
                                    "Autocorrelación Parcial (PACF) - Determina AR(p)"),
                    vertical_spacing=0.15)

# ACF

fig.add_trace(go.Bar(
    x=list(range(1, lags+1)), y=acf_vals,
    name='ACF', marker_color='rgb(31, 119, 180)', showlegend=False
), row=1, col=1)

# Intervalos de confianza (Sombreado)
fig.add_shape(type="rect",
    x0=0.5, y0=-conf_interval, x1=lags+0.5, y1=conf_interval,
    line=dict(color="rgba(0,0,0,0)"), fillcolor="rgba(0,0,0,0.1)",
    row=1, col=1
)
# Líneas límite
fig.add_hline(y=conf_interval, line_dash="dash", line_color="gray", row=1, col=1)
fig.add_hline(y=-conf_interval, line_dash="dash", line_color="gray", row=1, col=1)

# PACF

fig.add_trace(go.Bar(
    x=list(range(1, lags+1)), y=pacf_vals,
    name='PACF', marker_color='rgb(255, 127, 14)', showlegend=False
), row=2, col=1)

# Intervalos de confianza
fig.add_shape(type="rect",
    x0=0.5, y0=-conf_interval, x1=lags+0.5, y1=conf_interval,
    line=dict(color="rgba(0,0,0,0)"), fillcolor="rgba(0,0,0,0.1)",
    row=2, col=1
)
# Líneas límite
fig.add_hline(y=conf_interval, line_dash="dash", line_color="gray", row=2, col=1)
fig.add_hline(y=-conf_interval, line_dash="dash", line_color="gray", row=2, col=1)


fig.update_layout(
    title='<b>Diagnóstico de Estructura: ACF y PACF</b><br><sup>Serie Diferenciada</sup>',
    template='plotly_white',
    height=700,
    bargap=0.8 # Barras delgadas estilo lollipop
)

# Resaltar lags estacionales (7, 14, 21, 28) con líneas verticales rojas tenues
for i in [7, 14, 21, 28]:
    fig.add_vline(x=i, line_width=1, line_dash="dot", line_color="red", opacity=0.5)

fig.show()

The differentiated series to define the seasonal parameters of SARIMA seems to suggest $P=1$ and $Q=1$, with no seasonal parameter. 

# <font color= #99C8F5> **SARIMA Model** </font>

Since the ACF and PACF visualizations indicated that our time series is not ideal for a SARIMA Model, to the point of being unable to discern proper parameters for it, the parameter selection was done on a test basis, until we eventually landed on this model order:

$$
SARIMA(3, 0, 1)(0, 0, 1)_7
$$

In [40]:
# Drop team column, since it has no predictive power
goals_per_day.drop(columns='team', inplace=True)

# Check for NaNs
pd.DataFrame(goals_per_day.isna().sum(), columns= ['null_values'])

,null_values
date,0
total_goals,0


In [69]:
TEST_DAYS = 30

DATE_COL = "date"        
Y_COL    = "total_goals"  

df = goals_per_day.copy()

df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors="coerce")
df = df.dropna(subset=[DATE_COL]).sort_values(DATE_COL).set_index(DATE_COL)

df[Y_COL] = pd.to_numeric(df[Y_COL], errors="coerce")
df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=[Y_COL])

df = df.asfreq("D")                 
df[Y_COL] = df[Y_COL].fillna(0)     

y = df[Y_COL]  

train = y.iloc[:-TEST_DAYS]
test  = y.iloc[-TEST_DAYS:]

model = SARIMAX(
    train,
    order=(3,0,1),
    seasonal_order=(0,1,1,7),
    enforce_stationarity=False,
    enforce_invertibility=False
)

results = model.fit(disp=False)

forecast_object = results.get_forecast(steps=len(test))
forecast_vals = forecast_object.predicted_mean
conf_int = forecast_object.conf_int(alpha=0.05)

forecast_vals = forecast_vals.reindex(test.index)

rmse = np.sqrt(mean_squared_error(test, forecast_vals))
mape = mean_absolute_percentage_error(test, forecast_vals)  
mae  = mean_absolute_error(test, forecast_vals)

print(f"\n--- Errores del modelo ---")
print(f"RMSE: {rmse:.2f} Goals")
print(f"MAPE: {mape:.2%}")
print(f"MAE: {mae:.2f} Goals")

print(results.summary())


--- Errores del modelo ---
RMSE: 0.80 Goals
MAPE: 59864505465024152.00%
MAE: 0.41 Goals
                                     SARIMAX Results                                     
Dep. Variable:                       total_goals   No. Observations:                  832
Model:             SARIMAX(3, 0, 1)x(0, 1, 1, 7)   Log Likelihood                -762.052
Date:                           Sun, 22 Feb 2026   AIC                           1536.104
Time:                                   20:29:39   BIC                           1564.330
Sample:                               08-12-2023   HQIC                          1546.937
                                    - 11-20-2025                                         
Covariance Type:                             opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.9174      0.185 

In [70]:
# Grafica
fig = go.Figure()

# Train
fig.add_trace(go.Scatter(
    x=train.index, y=train,
    mode='lines',
    name='Train',
    line=dict(color='rgba(100, 100, 100, 0.6)', width=1.5)
))

# Test
fig.add_trace(go.Scatter(
    x=test.index, y=test,
    name='Test',
    line=dict(color='#1f77b4', width=3),
    marker=dict(size=6)
))

# Forecast
fig.add_trace(go.Scatter(
    x=test.index, y=forecast_vals,
    name='SARIMA',
    line=dict(color='#ff7f0e', width=3, dash='dot')
))

# Intervalos de Confianza
fig.add_trace(go.Scatter(
    x=conf_int.index, y=conf_int.iloc[:, 0],
    mode='lines', line=dict(width=0), showlegend=False, hoverinfo='skip'
))
fig.add_trace(go.Scatter(
    x=conf_int.index, y=conf_int.iloc[:, 1],
    mode='lines', line=dict(width=0), fill='tonexty',
    fillcolor='rgba(255, 127, 14, 0.2)',
    name='Int. Confianza 95%', hoverinfo='skip'
))

# Titulos
fig.update_layout(
    title=f'<b>Modelo SARIMA: Pronóstico de Carreras MLB</b><br><sup>',
    xaxis_title='Fecha',
    yaxis_title='Total de Carreras',
    legend=dict(x=0, y=1, bgcolor='rgba(255,255,255,0.8)'),
    hovermode="x unified"
)

fig.show()

The series doesn't show a clear pattern, which makes it hard for the model to adjust correctly to the data. Additionally, since the missing values in the series suggest that 0 goals where scored, the model predicts 0 goals.

# <font color= #99C8F5> **Conclusions** </font>

The selected time series data was not the best one to model using a SARIMA model, since there wasn't a clear seasonal pattern and the data was murky to begin with. Perhaps another soccer metric, such as player performance, instead of goals scored would have been a better predictor. However, there seems to be an absence of free APIs for soccer specifically, since most of them ask for payments if you want to access more useful data for each match.

This suggests that modeling data for soccer is more complicated than it seems, both for the data available and the paywall behind it. Even then, a half-decent time series was achieved, although by trial and error instead of selecting the order based on the autocorrelation functions.